## Code for calculating personalized FSA score

This is a demo for generating FSA score in our dataset. Before this step, we need to first preprocess our fMRI data (see ReadMe) and then get preprocessed fMRI image and corresponding fALFF image for each patient.

Then use the following script to load our data and materials.

In [1]:
from nilearn import image, masking
import copy
import numpy as np
from scipy import stats, ndimage
import os

# load datasets 
fmri_973 = datasets.fetch_973_fmri(center=1)
vbf_973 = datasets.fetch_973_vbf(center=1)
subjects = [s for s in vbf_973['subject_index'] if s[:2] == 'SZ' or s[:2] == 'NC'][:50]
vbf = np.array(vbf_973['ALFF_GR'])[[vbf_973['subject_index'].index(s) for s in subjects]]
fmris = np.array(fmri_973['fMRI_GR'])[[fmri_973['subject_index'].index(s) for s in subjects]]
sample = image.load_img(fmris[0])

#load materials
mask = image.load_img('../mask_ICV_WB.nii')
tem6 = image.load_img('../f6.nii')
tem8 = image.load_img('../f8.nii')
seed_striatum = image.load_img('../striatum.nii')

# resample  
seed_striatum_res = image.resample_to_img(seed_striatum, sample, interpolation='nearest')
mask_res = image.resample_to_img(mask, sample, interpolation='nearest')
mask_tem6 = image.resample_to_img(mask, tem6, interpolation='nearest')
striatum_mask_res6 = image.resample_to_img(seed_striatum, tem6, interpolation='nearest')
striatum_mask_res8 = image.resample_to_img(seed_striatum, tem8, interpolation='nearest')
striatum_index = masking.apply_mask(seed_striatum_res, mask_res)
striatum_mask_res_data = masking.apply_mask(striatum_mask_res6, mask_tem6)

/DATA/237/ali/Software/anaconda2/lib/python2.7/site-packages/scipy/ndimage/interpolation.py:478: UserWarning: The behaviour of affine_transform with a one-dimensional array supplied for the matrix parameter has changed in scipy 0.18.0.
  "The behaviour of affine_transform with a one-dimensional "


Next, we extracted striatal features from fMRI&fALFF images.

Notably, to balance sample size and feature number, we computed intra-striatum FC in the resolution of 8 mm and extra-striatum FC in 6 mm. 

In [3]:
import joblib

save_path = '../FSA_candidates'
def generate_FSA_candidates(name, fmris, vbf, subjects):
    
    """
    A function that parallelly extracting each individual's fMRI features to 
    further compute FSA score 
    
    """
    
    for n, (fmri, falff, s) in enumerate(zip(fmris, vbf, subjects)):
        f_file = fmri
        f_img_uns = image.load_img(f_file)
        # applying smooth when using voxel-wise features
        f_img = image.smooth_img(f_img_uns, fwhm=6)
        fmri_data_uns = masking.apply_mask(f_img_uns, mask_res)
        fmri_data = masking.apply_mask(f_img, mask_res)
        fmri_data[np.isnan(fmri_data)] = 0
        
        # extracting extra-striatum FC
        ts_striatum = np.mean(fmri_data_uns * striatum_index, axis=1)
        corr_striatum = np.zeros_like(striatum_index)
        for i in range(fmri_data.shape[1]):
            corr_striatum[i] = stats.pearsonr(ts_striatum, fmri_data[:, i])[0]
        corr_img = masking.unmask(corr_striatum, mask_res)
        corr_img_tem6 = image.resample_to_img(corr_img, tem6)
        str_corr = masking.apply_mask(corr_img_tem6, mask_tem6)
        str_corr[np.isnan(str_corr)] = 0
        corr_striatum_other = str_corr[striatum_mask_res_data == 0]

        # extracting intra-striatum FC
        f_img_res = image.resample_to_img(f_img, tem8)
        ts_str = masking.apply_mask(f_img_res, striatum_mask_res8)
        corr = np.corrcoef(ts_str.T)
        fc_str = corr[np.tril_indices_from(corr, -1)]
        # extracting fALFF
        striatum_seed_res = image.resample_to_img(target_img=falff, source_img=seed_striatum, interpolation='nearest')
        alff = masking.apply_mask(falff, mask_img=striatum_seed_res)

        fc = np.concatenate([alff, corr_striatum_other, fc_str])
        print('Run task %s (%s)...' % (name, os.getpid()))
        joblib.dump(fc, os.path.join(save_path, s + '_candidates.pkl'))
        
if __name__=='__main__':
    print('Parent process %s.' % os.getpid())
    kernels = 25
    batch = len(subjects) / kernels
    p = Pool(kernels)
    subs = [[sub for sub in subjects[batch * i: batch * i + batch]] for i in range(kernels)]
    fmri_files = [[f for f in fmris[batch * i: batch * i + batch]] for i in range(kernels)]
    vbf_files = [[f for f in vbf[batch * i: batch * i +batch]] for i in range(kernels)]
    for i in range(kernels):
        p.apply_async(generate_FSA_candidates, args=(i, fmri_files[i], vbf_files[i], subs[i]))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')

Parent process 24021.
Waiting for all subprocesses done...
Run task 21 (24746)...
Run task 1 (24726)...
Run task 19 (24744)...
Run task 13 (24738)...
Run task 8 (24733)...
Run task 4 (24729)...
Run task 17 (24742)...
Run task 14 (24739)...
Run task 0 (24725)...
Run task 3 (24728)...
Run task 11 (24736)...
Run task 9 (24734)...
Run task 2 (24727)...
Run task 10 (24735)...
Run task 12 (24737)...
Run task 15 (24740)...
Run task 23 (24748)...
Run task 20 (24745)...
Run task 24 (24749)...
Run task 18 (24743)...
Run task 16 (24741)...
Run task 22 (24747)...
Run task 5 (24730)...
Run task 7 (24732)...
Run task 6 (24731)...
Run task 8 (24733)...
Run task 14 (24739)...
Run task 17 (24742)...
Run task 11 (24736)...
Run task 4 (24729)...
Run task 21 (24746)...
Run task 9 (24734)...
Run task 13 (24738)...
Run task 10 (24735)...
Run task 0 (24725)...
Run task 20 (24745)...
Run task 1 (24726)...
Run task 24 (24749)...
Run task 19 (24744)...
Run task 3 (24728)...
Run task 23 (24748)...
Run task 12 (2

Finally, the personalized FSA score was computed as each individual's distance to SVM hyperplane. 

In [20]:
import joblib
from sklearn.metrics import accuracy_score

# load precomputed striatal features 
fc = np.zeros([len(subjects), 12689])
for i, s in enumerate(subjects):
    fc[i, :] = joblib.load(os.path.join(save_path, s + '_candidates.pkl'))

# load pre-trained model of standardizing features by all samples from 7 sites
prep = joblib.load('../model_pre_final.m')

# load pre-trained model trained by all samples from 7 sites
svc = joblib.load('../model_final.m')
predict = svc.predict(prep.transform(fc))
test = np.array([1 if s[:2] == 'NC' else -1 for s in subjects])
print accuracy_score(predict, test)
FSA = svc.decision_function(prep.transform(fc))

0.96
[ 1.0253824   1.02322132 -0.98628936  0.3041096   0.99673155 -0.88249479
  0.98898912 -0.98016657  1.40044007  1.00739396  1.01340555  1.39798684
 -0.96361039 -1.01322722 -1.01986687 -0.98245666 -0.99815214 -1.0231223
 -1.01360945  1.10266417  0.89289263  1.0088927  -0.99235743  0.98389349
  1.83938481  0.54326621 -0.99429394 -1.3834942  -0.98599612  0.98531606
 -1.22047699 -0.42285164  1.01983302  0.99141984  1.00293613 -0.33026515
 -0.99218734  0.94209741  0.98268963  1.02264916  1.00687717  1.00732661
  0.98543159  1.01866249 -0.96500253  0.67487582 -0.99472199 -0.98630546
 -0.98639421  0.22021234]
